## 1. A description of the problem and a discussion of the background.

### Scenario:

Currently I live in Amsterdam where is the captical city in Netherlands and everyday I need to drive more than oen hour to my work(Rotterdam)where is on the other side of the country.Traffic jams in Netherlands increases during both the morning and evening rush hour as well as during the day. In order to save commuting time, I decide to relocate to a new neighbourhood where is also close to my office. The queation is: I would like to find a cosy and convenient place similar to my currect house. Since I will move together with my family, the new place would be most desirable in the neighbourhood with sport center, school supermarkets and restaurants. 

Some requirements are shown as below before I can finally make a comparison and evaluation of the new place to live:

 * house must have 3 bedrooms and garden
 * desired location is near a bus station or a metro station within 2km 
 * house price is between 300.000 and 350.000 euro
 * desirable to have venues such as coffee shops, restaurants,gym and food shops,schools
 * the selected neighborhood shall be similar to current residence

## Business Problem

The netherlands is a small country but with high-density population, the house price has been dramaticlly increased in the last few years especially in the area of randstad. Therefore it is always a big challenge for renting/buying in big cities such as Amsterdam, Rotterdam, Den Haag and Utrecht. The final goal of this project is aim to find a suitable neighbourhood with demands on location, price and environment, which has similarities as the currect living.

# 2. A description of the data and how it will be used to solve the problem

## Data section

The following data is required for solving this problem:
  
  * List of available houses in Rotterdam with address and price
  * List of boroughs and neighbourhood of Rotterdam with its geodata
  * List of schools in Rotterdam with location 
  * Venues of each desired neighbourhood and stations

## How the data can be used to solve the problem

  * Use Foursquare and geopy data to map top 10 venues for all Rotterdam neighborhoods and clustered in groups 
  * Use foursquare and geopy data to map the location of bus stations , separately and on top of the above clustered map in order to be able to identify the venues near each station, or explore each subway location separately
  * Use Foursquare and geopy data to map the location of houses.
  * Create a map that depicts, for instance, the average house price per square meter, around a radious of 2km around each station - or a similar metrics. 
  * Addresses from houses will be converted to geodata( latitude, longtitude) using Geopy-distance and Nominatim.


The procesing of these DATA will allow to answer the key questions to make a decision:

 * what is the cost of house(per square meter) around a specific radius from each station?
 * what is the area of Rotterdam with best house pricing that meets criteria established?
 * What is the distance between the work place and the tentative future home?
 * What are the venues of the two best places to live? How the prices compare?
 * How venues distribute among Rotterdam neighborhoods and around supermarkets?
 * Are there tradeoffs between size and price and location?


### Reference of venues around current residence in Amsterdam for comparison to Rotterdam 

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
CLIENT_ID = 'CP5 *****************BU' # your Foursquare ID
CLIENT_SECRET = 'TA****************1C' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
 

In [4]:
address = 'Rijnstraat, Amsterdam, NL'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.3406123 4.9072881


In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CP5HSTNJ4NMVZWJAYXRX24GHVPSWHSKJZ1MR5WSCY1DHGCBU&client_secret=TAJQ5VGIEYJOGBLGN2UEDOT0HYRGC4PZQV3ZGTAXS4VIMG1C&v=20180604&ll=52.3406123,4.9072881&radius=1000&limit=100'

In [6]:
results = requests.get(url).json()


In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
 

In [10]:
venues = results['response']['groups'][0]['items']
    
AMSnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
AMSnearby_venues =AMSnearby_venues.loc[:, filtered_columns]

# filter the category for each row
AMSnearby_venues['venue.categories'] = AMSnearby_venues.apply(get_category_type, axis=1)

# clean columns
AMSnearby_venues.columns = [col.split(".")[-1] for col in AMSnearby_venues.columns]

AMSnearby_venues.head(10)

,name,categories,lat,lng
0,IJssalon IJspret,Ice Cream Shop,52.342430,4.906680
1,De Parade,Circus,52.340436,4.910966
2,Thuis aan de Amstel,Café,52.339639,4.914769
3,Kong Kha,Thai Restaurant,52.345182,4.905655
4,De Mirandabad,Pool,52.338856,4.901854
5,Eatalia Gastronomia,Deli / Bodega,52.342951,4.906738
6,Fixables,Electronics Store,52.342474,4.906724
7,Betty's,Vegetarian / Vegan Restaurant,52.345541,4.905541
8,Alles Voor De Lunch,Breakfast Spot,52.343097,4.907701
9,Hi-Fi Klubben,Electronics Store,52.343084,4.906362


In [12]:
# create map of Singapore place  using latitude and longitude values
map_AMS = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(AMSnearby_venues['lat'], AMSnearby_venues['lng'], AMSnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_AMS